<img src="assets/strathsdr_banner.png" align="left">

# Modulation Classification with RFSoC with Post-Quantisation Trained (PTQ) Weights

## Overview
Deploying **4** CNN models each with different weight fixed-point widths **(16-bit, 8-bit, 4-bit, and 2-bit)**. All 4 models are operating on live signals in real-time and at the same time.

## Signal Specifications
Option of **8** modulation schemes: **QPSK, BPSK, QAM16, QAM64, PSK8, PAM4, GFSK, CPFSK**.

**Sampling Rate**: 128 MSPS

**Decimation/Interpolation Rate**: 32

## Throughput and Latency for each Model
*With PE unrolling along the number of filters (N) dimension*

**Throughput**: 34k classifications per second

**Latency**: 29.6$\mu s$

# Post-Training Quantisation (PTQ)

In this FPGA bitstream, each of the models have used weights quantised after training. The resulting loss plots and weight value distributions for each model are shown.

<div style="display: flex; justify-content: center; gap: 20px; flex-wrap: wrap;">
  <figure style="flex: 1 1 45%; margin: 0; text-align: center;">
    <img src="./assets/loss_deeprfsoc.svg" style="width: 100%; height: auto;"/>
    <figcaption><b>Figure 1: Training loss plots.</b></figcaption>
  </figure>

  <figure style="flex: 1 1 45%; margin: 0; text-align: center;">
    <img src="./assets/ptq_weight_dist.svg" style="width: 80%; height: auto;"/>
    <figcaption><b>Figure 2: Quantised weight value distribution.</b></figcaption>
  </figure>
</div>


# FPGA Programmable Logic Configuration
The FPGA is configured in a typical radio receiver setup where a signals is received at the ADC and samples are streamed into the radio. The samples are decimated and processed by the deployed model to determine the modulation scheme of the signal.

<figure>
<img src='./assets/experimentalSetup.svg' width='80%'/>
<figcaption><b>Figure 1: The architecture overview of the RFSoC classifying signals.</b></figcaption>
</figure>

# PTQ Weight Overall Accuracies
The accuracies achieved when quantising weights after training results in very different curves for all quantisation widths, despite the reduction in available precision.

<figure>
<img src='./assets/ptq_loopback_accuracy.svg' width='60%'/>
<figcaption><b>Figure 1: Accuracy of each model deployed with models using PTQ.</b></figcaption>
</figure>

In [ ]:
from rfsoc_quant_amc.overlay import Overlay

ol = Overlay(model='ptq')

In [ ]:
from rfsoc_quant_amc import helper as h

filename = 'transmit_test_SNR.pkl'
dataset = h.load_dataset(filename)

# Inspect an example
mod = 'QPSK'
snr = '30'
data_mod = dataset[mod,snr][:,:,0]
h.plot_dataset(data_mod)

In [ ]:
ADC_SAMPLING_FREQ = 1024.0 # MHz
ADC_PLL_FREQ = 409.6 # MHz
centre_freq = 400 # MHz

ol.initialise_adcs(pll_freq=ADC_PLL_FREQ, 
                   sampling_freq=ADC_SAMPLING_FREQ, 
                   centre_freq=centre_freq)

In [ ]:
DAC_SAMPLING_FREQ = 1024.0 # MHz
DAC_PLL_FREQ = 409.6 # MHz
centre_freq = 400 # MHz

ol.initialise_dacs(pll_freq=DAC_PLL_FREQ, 
                   sampling_freq=DAC_SAMPLING_FREQ, 
                   centre_freq=centre_freq)

In [ ]:
ol.initialise_ips()

In [ ]:
import numpy as np
mods = ['QPSK','BPSK','QAM16','QAM64','PSK8','PAM4','GFSK','CPFSK']
snrs = ['-20', '-16', '-12', '-8', '-4', '0', '4', '8', '12', '16', '20', '24', '28','30']

mod = 'QPSK' # Choose from the selection of modulation schemes
snr = '30'   # Choose from the selection of SNRs
data_mod = dataset[mod,snr][:,:,0]
# Scale to fit DAC range
y = np.int16(data_mod*np.int16(pow(2,13)))
# Interleave complex samples
z = np.zeros(2*4096, dtype=np.int16)
z[0::2] = y[0,:]
z[1::2] = y[1,:]
# Send to DAC
ol.send(z)

In [ ]:
[y_pred_0, y_pred_1, y_pred_2, y_pred_3, re_data, im_data] = ol.receive_data()
print(f"Model 16w16a predicted {mods[y_pred_0]}")
print(f"Model  8w16a predicted {mods[y_pred_1]}")
print(f"Model  4w16a predicted {mods[y_pred_2]}")
print(f"Model  4w16a predicted {mods[y_pred_3]}")

In [ ]:
h.plot_received(re_data, im_data)

In [ ]:
# from rfsoc_quant_amc.new_widget import AMCWidget
from rfsoc_quant_amc.widget import AMCWidget
widget = AMCWidget(ol, dataset, mods, snrs)
widget.display()

---